In [80]:
# Update sbml_annotation so that specificity is weighted based on qualifier
import itertools
import libsbml
import networkx as nx
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import sys
sys.path.append(os.path.join(os.getcwd(),"" '..'))
from SBMate import constants as cn
from SBMate import metric_calculator as mc
from SBMate import sbmate
from SBMate import sbml_annotation as sa

from SBMate import dag_analyzer as da
from SBMate import uniprot_kegg_analyzer as uka
BIOMD_12 = 'BIOMD0000000012.xml'
BIOMD_13 = 'BIOMD0000000013.xml'
BIOMD_15 = 'BIOMD0000000015.xml'
BIOMD_970 = 'BIOMD0000000970.xml'

In [2]:
raw_annotation = sa.RawSBMLAnnotation(BIOMD_12)

In [3]:
a = dict()
if a:
  print("a is dict")
print(type(a))

<class 'dict'>


In [4]:
for one_anot in raw_annotation.str_annotation[:3]:
    print(one_anot)
    print()

ObjectAnnotation(id='BIOMD0000000012', object_type=<class 'libsbml.Model'>, annotation={'isVersionOf': ['<bqbiol:isVersionOf>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0040029"/>\n        </rdf:Bag>\n      </bqbiol:isVersionOf>']})

ObjectAnnotation(id='cell', object_type=<class 'libsbml.Compartment'>, annotation={'is': ['<bqbiol:is>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0005623"/>\n        </rdf:Bag>\n      </bqbiol:is>']})

ObjectAnnotation(id='PX', object_type=<class 'libsbml.Species'>, annotation={'is': ['<bqbiol:is>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/uniprot/P03023"/>\n        </rdf:Bag>\n      </bqbiol:is>']})



In [5]:
processed_annotation = sa.SBMLAnnotation(BIOMD_12)

In [6]:
processed_annotation.annotations['PX']

{'go': None,
 'sbo': ['SBO:0000252'],
 'chebi': None,
 'kegg_species': None,
 'kegg_process': None,
 'uniprot': ['P03023'],
 'object_id': 'PX',
 'object_type': libsbml.Species}

### Test script (test_sbml_annotation2.py)

In [7]:
reader = libsbml.SBMLReader()
document = reader.readSBML(os.path.join(cn.TEST_DIR, BIOMD_12))
sbml_model = document.getModel()
raw_sbml_annotation = sa.RawSBMLAnnotation(input_file=os.path.join(cn.TEST_DIR, BIOMD_12))
# BIOMD13 is for checking incompelte annotation 
incomplete_document = reader.readSBML(os.path.join(cn.TEST_DIR, BIOMD_15))
incompelte_sbml_model = incomplete_document.getModel()
incomplete_raw_sbml_annotation = sa.RawSBMLAnnotation(input_file=os.path.join(cn.TEST_DIR, BIOMD_15))

In [8]:
# testGetSBOAnnotation
# reaction1_sbo_annotation = raw_sbml_annotation.getSBOAnnotation(sbml_model.getReaction('Reaction1'))
# print(reaction1_sbo_annotation.id)
# print(reaction1_sbo_annotation.object_type)
# print(reaction1_sbo_annotation.annotation)

# testGetOntAnnotation
reaction1_annotation = raw_sbml_annotation.getOntAnnotation(sbml_model.getReaction('Reaction1'))
reaction1_str_annotation = '<bqbiol:isVersionOf>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0006402"/>\n        </rdf:Bag>\n      </bqbiol:isVersionOf>'
print(reaction1_annotation[0]=='Reaction1')
print(reaction1_annotation[1]==libsbml.Reaction)
print(reaction1_annotation[2]['isVersionOf'][0]==reaction1_str_annotation)

True
True
True


In [9]:
px_annotation = raw_sbml_annotation.getOntAnnotation(sbml_model.getSpecies('PX'))
px_str_annotation = '<bqbiol:is>\n  <rdf:Bag>\n    <rdf:li rdf:resource="http://identifiers.org/uniprot/P03023"/>\n  </rdf:Bag>\n</bqbiol:is>'
print(px_annotation.id=='PX')
print(px_annotation.object_type==libsbml.Species)
print(px_annotation.annotation['is'][0]==px_str_annotation)

True
True
False


In [10]:
no_ont_annotation = incomplete_raw_sbml_annotation.getOntAnnotation(incompelte_sbml_model.getSpecies('ATP'))
print(no_ont_annotation.id=='ATP')
print(no_ont_annotation.object_type==libsbml.Species)
print(no_ont_annotation.annotation=={}) 

True
True
True


In [11]:
sbml_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_12))
input_annotation = '<bqbiol:isVersionOf>\n  <rdf:Bag>\n    <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0006402"/>\n  </rdf:Bag>\n</bqbiol:isVersionOf>'
    

In [12]:
model_annotation = sbml_annotation.getAnnotationDictByQualifier('BIOMD0000000012')

In [13]:
model_annotation

{'isVersionOf': [('obo.go', 'GO:0040029')],
 'object_id': 'BIOMD0000000012',
 'object_type': libsbml.Model}

In [14]:
px_annotation = sbml_annotation.getAnnotationDictByQualifier('PX')
px_annotation

{'is': [('uniprot', 'P03023'), ('sbo', 'SBO:0000252')],
 'object_id': 'PX',
 'object_type': libsbml.Species}

In [15]:
model_anot_by_ont = sbml_annotation.getAnnotationDictByOntology(model_annotation)
px_anot_by_ont = sbml_annotation.getAnnotationDictByOntology(px_annotation)

In [16]:
model_anot_by_ont

{'go': ['GO:0040029'],
 'sbo': None,
 'chebi': None,
 'kegg_species': None,
 'kegg_process': None,
 'uniprot': None,
 'object_id': 'BIOMD0000000012',
 'object_type': libsbml.Model}

In [17]:
px_anot_by_ont

{'go': None,
 'sbo': ['SBO:0000252'],
 'chebi': None,
 'kegg_species': None,
 'kegg_process': None,
 'uniprot': ['P03023'],
 'object_id': 'PX',
 'object_type': libsbml.Species}

In [18]:
sbml_annotation.getKnowledgeResourceTuple('input_annotation')

[]

### Work on MetricCalculator

In [19]:
metrical = mc.MetricCalculator(annotations=sbml_annotation, model_name='BIOMD0000000012')

In [20]:
metrical

In [21]:
one_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_12))
two_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_970))
calculator = mc.MetricCalculator(annotations=one_annotation, model_name=BIOMD_12)
none_calculator = mc.MetricCalculator(annotations=two_annotation, model_name=BIOMD_970)

In [22]:
metrics_df = calculator.calculate()
print(metrics_df.index[0] == BIOMD_12)
# self.assertEqual(metrics_df.shape, (1,6))
# self.assertEqual(int(metrics_df['annotatable_elements']), 20)
# self.assertEqual(int(metrics_df['annotated_elements']), 20)
# self.assertEqual(float(metrics_df['coverage']), 1.0)
# self.assertEqual(int(metrics_df['consistent_elements']), 19)
# self.assertEqual(float(metrics_df['consistency']), 0.95)
# self.assertEqual(float(metrics_df['specificity']), 0.88)

True


In [23]:
metrics_df

,annotatable_elements,annotated_elements,coverage,consistent_elements,consistency,specificity
BIOMD0000000012.xml,20,20,1.0,19,0.95,0.7


In [31]:
# Check that, whether specificity is correct; 

In [29]:
print(list(sbml_annotation.annotations.keys()))

['BIOMD0000000012', 'cell', 'PX', 'PY', 'PZ', 'X', 'Y', 'Z', 'Reaction1', 'Reaction2', 'Reaction3', 'Reaction4', 'Reaction5', 'Reaction6', 'Reaction7', 'Reaction8', 'Reaction9', 'Reaction10', 'Reaction11', 'Reaction12']


In [50]:
metrical.annotations.annotations['PX']

{'go': None,
 'sbo': ['SBO:0000252'],
 'chebi': None,
 'kegg_species': None,
 'kegg_process': None,
 'uniprot': ['P03023'],
 'object_id': 'PX',
 'object_type': libsbml.Species}

In [33]:
consistent_entitics, _ = calculator._getConsistency(list(sbml_annotation.annotations.keys()))

In [35]:
consistent_entitics.keys()

dict_keys(['BIOMD0000000012', 'PX', 'PY', 'PZ', 'X', 'Y', 'Z', 'Reaction1', 'Reaction2', 'Reaction3', 'Reaction4', 'Reaction5', 'Reaction6', 'Reaction7', 'Reaction8', 'Reaction9', 'Reaction10', 'Reaction11', 'Reaction12'])

In [57]:
anot_key = 'PX'
specificities = []
for anot_key in consistent_entitics.keys():
  one_anot = calculator.annotations.annotations[anot_key]
  one_anot_qualifier = calculator.annotations.annotation_by_qualifier[anot_key]
  term_qualifier_map = calculator.getQualifierDict(one_anot_qualifier)
  one_entity_onts = [mc.ANALYZER_DICT[key](term_id=one_anot[key],
                                    ontology=key,
                                    object_type=one_anot['object_type'],
                                    qualifier_dict=term_qualifier_map) \
                  for key in mc.ANALYZER_DICT if one_anot[key]]
  specificities.append((anot_key, calculator._getSpecificity({anot_key: one_entity_onts})))
print(specificities)

[('BIOMD0000000012', 0.35), ('PX', 1.0), ('PY', 1.0), ('PZ', 1.0), ('X', 0.65), ('Y', 0.65), ('Z', 0.65), ('Reaction1', 0.67), ('Reaction2', 0.67), ('Reaction3', 0.67), ('Reaction4', 0.7), ('Reaction5', 0.7), ('Reaction6', 0.7), ('Reaction7', 0.65), ('Reaction8', 0.65), ('Reaction9', 0.65), ('Reaction10', 0.66), ('Reaction11', 0.66), ('Reaction12', 0.66)]


In [84]:
for anot_key in consistent_entitics.keys():
  print(anot_key)
  print(calculator.annotations.annotation_by_qualifier[anot_key])

BIOMD0000000012
{'isVersionOf': [('obo.go', 'GO:0040029')], 'object_id': 'BIOMD0000000012', 'object_type': <class 'libsbml.Model'>}
PX
{'is': [('uniprot', 'P03023'), ('sbo', 'SBO:0000252')], 'object_id': 'PX', 'object_type': <class 'libsbml.Species'>}
PY
{'is': [('uniprot', 'P04483'), ('sbo', 'SBO:0000252')], 'object_id': 'PY', 'object_type': <class 'libsbml.Species'>}
PZ
{'is': [('uniprot', 'P03034'), ('sbo', 'SBO:0000252')], 'object_id': 'PZ', 'object_type': <class 'libsbml.Species'>}
X
{'isVersionOf': [('chebi', 'CHEBI:33699'), ('kegg.compound', 'C00046')], 'is': [('sbo', 'SBO:0000250')], 'object_id': 'X', 'object_type': <class 'libsbml.Species'>}
Y
{'isVersionOf': [('chebi', 'CHEBI:33699'), ('kegg.compound', 'C00046')], 'is': [('sbo', 'SBO:0000250')], 'object_id': 'Y', 'object_type': <class 'libsbml.Species'>}
Z
{'isVersionOf': [('chebi', 'CHEBI:33699'), ('kegg.compound', 'C00046')], 'is': [('sbo', 'SBO:0000250')], 'object_id': 'Z', 'object_type': <class 'libsbml.Species'>}
Reactio

In [65]:
anot_key = 'X'
one_anot = calculator.annotations.annotations[anot_key]
one_anot_qualifier = calculator.annotations.annotation_by_qualifier[anot_key]
term_qualifier_map = calculator.getQualifierDict(one_anot_qualifier)
one_entity_onts = [mc.ANALYZER_DICT[key](term_id=one_anot[key],
                                    ontology=key,
                                    object_type=one_anot['object_type'],
                                    qualifier_dict=term_qualifier_map) \
                  for key in mc.ANALYZER_DICT if one_anot[key]]
one_entity_onts

In [66]:
term_qualifier_map

{'SBO:0000250': 'is', 'CHEBI:33699': 'isVersionOf', 'C00046': 'isVersionOf'}

In [76]:
one_sub_ent_onts = one_entity_onts[0]
one_sub_ent_onts.consistent
print(one_sub_ent_onts.getOneTermSpecificity('SBO:0000250'))
print(one_sub_ent_onts.getSpecificity(['SBO:0000250']))


one_sub_ent_onts = one_entity_onts[1]
one_sub_ent_onts.consistent
print(one_sub_ent_onts.getOneTermSpecificity('CHEBI:33699'))
print(one_sub_ent_onts.getSpecificity(['CHEBI:33699']))

one_sub_ent_onts = one_entity_onts[2]
one_sub_ent_onts.consistent
print(one_sub_ent_onts.getSpecificity(['C00046']))

1.0
1.0
0.8829901743817397
0.44149508719086983
0.5


In [79]:
np.mean([1, 0.5, 0.44])

0.6466666666666666

In [ ]:
# Testing Analyzers, using the new Specificity rule. 

In [98]:
# DAG analyzer
sbml_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_12))
reaction1_annotation = sbml_annotation.annotations['Reaction1']
reaction1_analyzer = da.DAGAnalyzer(term_id=['GO:0006402'],
                                         ontology='go',
                                         object_type=libsbml.Reaction,
                                         qualifier_dict={'GO:0006402': 'isVersionOf'})

# def testFindRoot(self):
# print(reaction1_analyzer.findRoot(inp_term='GO:0006402') == 'GO:0008150')
# print(reaction1_analyzer.findRoot(inp_term='SBO:000123') == None)

# def testGetConsistency(self):
# print(reaction1_analyzer.getConsistency(inp_term=1.0))
# print(reaction1_analyzer.getConsistency(inp_term=['GO:0006402', 1]))
# print(reaction1_analyzer.getConsistency(inp_term=['GO:0006402']))
# print(reaction1_analyzer.getConsistency(inp_term=['GO:0110165']))

# def testGetOneTermSpecificity(self):
# dummy_analyzer = da.DAGAnalyzer(term_id=['SBO:0006402'],
#                                 ontology='go',
#                                 object_type=libsbml.Reaction,
#                                 qualifier_dict={'SBO:0006402': 'is'})
# # should return None if term is inconsistent
# print(dummy_analyzer.getOneTermSpecificity('SBO:0006402') == None)
# print(np.round(reaction1_analyzer.getOneTermSpecificity('GO:0006402'), 2) == 0.67)

# def testGetSpecificity(self):
dummy_analyzer1 = da.DAGAnalyzer(term_id=['SBO:0006402'],
                                 ontology='go',
                                 object_type=libsbml.Reaction,
                                qualifier_dict={'SBO:0006402': 'is'})
dummy_analyzer2 = da.DAGAnalyzer(term_id=['GO:0006402'],
                                 ontology='go',
                                 object_type=libsbml.Reaction,
                                qualifier_dict={'GO:0006402': 'isVersionOf'})
# should return None if term is inconsistent
print(dummy_analyzer1.getSpecificity(['SBO:0006402']) == None)
print(np.round(dummy_analyzer2.getSpecificity('GO:0006402'), 2) == 0.34)
print(np.round(dummy_analyzer2.getSpecificity(['GO:0006402']), 2) == 0.34)
print(dummy_analyzer2.getSpecificity(['GO:0006402', 'SBO:12345'])== None)

True
True
True
Calculate consistency first!
True


In [105]:
# Non-DAG analyzer
biomd12 = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_12))
px_annotation = biomd12.annotations['PX']
px_analyzer = uka.NonDAGAnalyzer(term_id=['P03023'],
                                 ontology='uniprot',
                                 object_type=libsbml.Species,
                                 qualifier_dict={'P03023': 'is'})
x_annotation = biomd12.annotations['X']
x_analyzer = uka.NonDAGAnalyzer(term_id=['C00046'],
                                ontology='kegg_species',
                                object_type=libsbml.Species,
                                qualifier_dict={'C00046': 'isVersionOf'})


#   def testGetOneTermConsistency(self):
# print(x_analyzer.getOneTermConsistency('C00046'))
# print(x_analyzer.getOneTermConsistency('GO:12345'))
# print(px_analyzer.getOneTermConsistency('P03023'))
# print(px_analyzer.getOneTermConsistency('GO:12345'))  

#   def testGetConsistency(self):
# print(x_analyzer.getConsistency(1.0))
# print(x_analyzer.getConsistency('GO:ABC'))
# print(x_analyzer.getConsistency(['C00046']))
# print(x_analyzer.getConsistency(['C00046', 1.0]))
# print(x_analyzer.getConsistency(['C00046', 'GO:ABC']))

#   def testGetSpecificity(self):
wrong_analyzer = uka.NonDAGAnalyzer(term_id=['GO:12345'],
                                    ontology='kegg_species',
                                    object_type=libsbml.Species,
                                    qualifier_dict={'GO:12345': 'is'})
print(wrong_analyzer.getSpecificity('GO:12345'), None)
print(x_analyzer.getSpecificity(['C00046']), 1.0)
print(px_analyzer.getSpecificity(['P03023']), 1.0)

None None
0.5 1.0
1.0 1.0


In [115]:
# Test for metric_calculator
one_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_12))
two_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_970))
calculator = mc.MetricCalculator(annotations=one_annotation, model_name=BIOMD_12)
none_calculator = mc.MetricCalculator(annotations=two_annotation, model_name=BIOMD_970)

# def testCalculate(self):
# metrics_df = calculator.calculate()
# print(metrics_df.index[0], BIOMD_12)
# print(metrics_df.shape, (1,6))
# print(int(metrics_df['annotatable_elements']), 20)
# print(int(metrics_df['annotated_elements']), 20)
# print(float(metrics_df['coverage']), 1.0)
# print(int(metrics_df['consistent_elements']), 19)
# print(float(metrics_df['consistency']), 0.95)
# print(float(metrics_df['specificity']), 0.70)
# print(float(metrics_df['specificity']) == 0.70)
# #
# non_metrics_df = none_calculator.calculate()
# print(non_metrics_df.index[0], BIOMD_970)
# print(non_metrics_df.shape, (1,6))
# print(int(non_metrics_df['annotatable_elements']), 10)
# print(non_metrics_df['annotated_elements'][0], None)
# print(float(non_metrics_df['coverage']), 0.0)
# print(non_metrics_df['consistent_elements'][0], None)
# print(non_metrics_df['consistency'][0], None)
# print(non_metrics_df['specificity'][0], None)

# def testMkDataframe(self):
# df = calculator.mkDataframe({'one_val': 10})
# print(df.shape, (1,1))
# print(df.index[0], BIOMD_12)
# print(int(df['one_val']), 10)

# def testGetCoverage(self):
# annotated_entities, coverage = calculator._getCoverage()
# print(len(annotated_entities), 20)
# print(coverage, 1.0)
# print(set(annotated_entities) == set(one_annotation.annotations.keys()))
# #
# none_annotated_entities, none_coverage = none_calculator._getCoverage()
# print(none_annotated_entities, [])
# print(none_coverage, 0.0)

# def testGetConsistency(self):
# annotated_entities, coverage = calculator._getCoverage()
# consistent_entities, consistency = calculator._getConsistency(annotated_entities)
# print(len(consistent_entities), 19)
# print(consistency, 0.95)
# print('cell' in set(consistent_entities))
# #
# none_annotated_entities, none_coverage = none_calculator._getCoverage()
# none_consistent_entities, none_consistency = none_calculator._getConsistency(none_annotated_entities)
# print(none_consistent_entities, None)
# print(none_consistency, None)

# def testGetSpecificity(self):
annotated_entities, coverage = calculator._getCoverage()
consistent_entities, consistency = calculator._getConsistency(annotated_entities)
specificity = calculator._getSpecificity(consistent_entities) 
print(specificity, 0.88)
#
none_annotated_entities, none_coverage = none_calculator._getCoverage()
none_consistent_entities, none_consistency = none_calculator._getConsistency(none_annotated_entities)
none_specificity = none_calculator._getSpecificity(none_consistent_entities)
print(none_specificity, None)

0.7 0.88
None None


In [166]:
# # test sbmate.py
# setup
import copy 

BIOMD_12 = 'BIOMD0000000012.xml'
BIOMD_13 = 'BIOMD0000000013.xml'
RESULT_REPORT_ENTRIES = [
    "annotatable_elements: 20",
    "annotated_elements: 20",
    "coverage: 1.0",
    "consistent_elements: 19",
    "consistency: 0.9",
    "specificity: 0.7",
    BIOMD_12,
    ]
IGNORE_TEST = False
IS_PLOT = False
MODEL_FILE = os.path.join(cn.TEST_DIR, BIOMD_12)
MODEL_FILE2 = os.path.join(cn.TEST_DIR, BIOMD_13)
ANNOTATION_METRICS = sbmate.AnnotationMetrics(MODEL_FILE)

METRIC_NAMES = [
    'annotatable_elements',
    'annotated_elements',
    'coverage',
    'consistent_elements',
    'consistency',
    'specificity',
    ]

def checkReport(report=None):
  if report is None:
    report = annotation_metrics._getMetricsReport()
  report = annotation_metrics._getMetricsReport()
  trues = [e in report for e in RESULT_REPORT_ENTRIES]
  print(all(trues))

In [167]:
print(all([True, True]))

True


In [168]:
#   def setUp(self):	
annotation_metrics = copy.deepcopy(ANNOTATION_METRICS)
no_annotation_metrics = sbmate.AnnotationMetrics()

#   def testInit(self):
df = annotation_metrics.metrics_df
print(no_annotation_metrics.annotations, None)
print(no_annotation_metrics.metrics_df, None)

None None
None None


In [169]:
annotation_metrics.metrics_df

,annotatable_elements,annotated_elements,coverage,consistent_elements,consistency,specificity
BIOMD0000000012.xml,20,20,1.0,19,0.95,0.7


In [162]:
report = annotation_metrics._getMetricsReport()

In [163]:
trues = [e in report for e in RESULT_REPORT_ENTRIES]
# self.assertTrue(all(trues))

In [164]:
trues

[True, True, True, True, True, False, True]

In [170]:
RESULT_REPORT_ENTRIES

['annotatable_elements: 20',
 'annotated_elements: 20',
 'coverage: 1.0',
 'consistent_elements: 19',
 'consistency: 0.9',
 'specificity: 0.7',
 'BIOMD0000000012.xml']

In [178]:
#   def testTwoClasses(self):
#     if IGNORE_TEST:
#       return
# annotation_metrics = sbmate.AnnotationMetrics(MODEL_FILE,
#                                               metric_calculator_classes=[mc.MetricCalculator])
# df = annotation_metrics.metrics_df
# lst = df.columns.tolist()
# for name in METRIC_NAMES:
#   print(lst.count(name), 2)

#   def basicChecks(self, df):
# print(df.shape, (1,6))
# print(df.index[0], BIOMD_12)
# print(int(df['annotatable_elements']), 20)
# print(int(df['annotated_elements']), 20)
# print(float(df['coverage']), 1.0)
# print(int(df['consistent_elements']), 19)
# print(float(df['consistency']), 0.95)
# print(float(df['specificity']), 0.70)

#   def testGetMetrics(self):
# res_report = sbmate.AnnotationMetrics.getMetrics(MODEL_FILE,
#     output="report")
# checkReport(report=res_report)

# res_df = annotation_metrics.getMetrics(MODEL_FILE,
#     output="table")
# print(res_df.shape, (1,6))
# print(int(res_df['annotatable_elements']), 20)
# print(int(res_df['annotated_elements']), 20)
# print(float(res_df['coverage']), 1.0)
# print(int(res_df['consistent_elements']), 19)
# print(float(res_df['consistency']), 0.95)
# print(float(res_df['specificity']), 0.7) 

# res_df2 = annotation_metrics.getMetrics([MODEL_FILE, MODEL_FILE2], 
#                                               output="table")
# print(res_df2.shape, (2,6))
# print(list(res_df2.columns), METRIC_NAMES)
# print(list(res_df2.loc[BIOMD_12,:]) ==
#                  [20, 20, 1.0, 19, 0.95, 0.7])
# print(list(res_df2.loc[BIOMD_13,:]) ==
#                  [51, 51, 1.0, 51, 1.00, 0.93])
    
res_report =  annotation_metrics.getMetrics(MODEL_FILE,
    output="report")
print(res_report == annotation_metrics._getMetricsReport())
# with self.assertRaises(ValueError) as context: 
#   self.annotation_metrics.getMetrics(123, output="table")
# with self.assertRaises(ValueError) as context: 
#   self.annotation_metrics.getMetrics([123, 'abc'], output="table")

# with self.assertRaisesRegex(ValueError, "Should be a valid file name."): 
#   self.annotation_metrics.getMetrics(123, output="table")  
# with self.assertRaisesRegex(ValueError, "Should be a valid file name."): 
#   self.annotation_metrics.getMetrics([123, 'abc'], output="table")  



#   def testGetMetricsReport(self):
#     if IGNORE_TEST:
#         return
#     self.checkReport()

#   def testMetricsTable(self):
#     if IGNORE_TEST:
#       return
df = annotation_metrics.metrics_df
print(df.shape, (1,6))
print(int(df['annotatable_elements']), 20)
print(int(df['annotated_elements']), 20)
print(float(df['coverage']), 1.0)
print(int(df['consistent_elements']), 19)
print(float(df['consistency']), 0.95)
print(float(df['specificity']), 0.7)

True
(1, 6) (1, 6)
20 20
20 20
1.0 1.0
19 19
0.95 0.95
0.7 0.7


In [153]:
annotation_metrics.getMetrics(MODEL_FILE,
    output="report")

'Summary of Metrics (BIOMD0000000012.xml)\n----------------------\nannotatable_elements: 20\nannotated_elements: 20\ncoverage: 1.0\nconsistent_elements: 19\nconsistency: 0.95\nspecificity: 0.7\n----------------------\n'

In [161]:
print(res_report)

Summary of Metrics (BIOMD0000000012.xml)
----------------------
annotatable_elements: 20
annotated_elements: 20
coverage: 1.0
consistent_elements: 19
consistency: 0.95
specificity: 0.7
----------------------

